In [50]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [63]:
df = pd.read_excel(r'ManualTest.xlsx', engine='openpyxl')

In [64]:
from api_helper import * 

In [65]:
def seq_question(context, answer):
    index = context.find(answer)
    if index != -1:
        answer_start = index
    else:
        answer_start = 0
    question = ''

    df = pd.DataFrame({'context':[context], 'question':[question], 'answer':[answer], 'answer_start':[answer_start]})
    dataframe = build_lexical_features(df)
    csv_path = 'test.csv'
    dataframe.to_csv(csv_path, index=False)
    test_dataset = TabularDataset(path=csv_path, format='csv', fields=fields, skip_header=True)
    
    for example in test_dataset.examples:
        src = vars(example)['context']
        ans = vars(example)['bio']
        lex = vars(example)['lex']
        question, logits = predict_question(model, src, ans, lex)
    
    return " ".join(question)


In [66]:
for i in range(1, 102):
    context = df.iloc[i][2]
    answer = df.iloc[i][3]
    question = seq_question(context, answer)
    df.at[i, 'Seq2seq'] = question

Artificial Intelligence (AI) has emerged as a transformative force, reshaping the landscape of technology and influencing various aspects of our lives.
1
As we navigate the 21st century, AI stands at the forefront of innovation, bringing forth new paradigms that redefine how we interact with machines and perceive the capabilities of modern technology. At its core, AI is a branch of computer science dedicated to creating intelligent machines capable of mimicking human cognitive functions.
1
This includes learning, problem-solving, and decision-making – processes that were traditionally exclusive to human intellect.
1
The evolution of AI has been marked by significant breakthroughs in machine learning, neural networks, and deep learning, enabling machines to process vast amounts of data and improve their performance over time.One of the key drivers of AI advancement is the paradigm shift towards decentralized computing and edge computing.
1
Traditionally, AI algorithms were confined to p

In [67]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Generated \nQuestions,Unnamed: 5,Human Criteria Results,Unnamed: 7,Seq2seq
0,NaN,NaN,Context,Answer,Custom,Pre-trained,Custom2,Pre-trained3,NaN
1,TOPIC 1,1.0,Artificial Intelligence (AI) has emerged as a ...,Artificial Intelligence,What is an example of an example of a result o...,What does AI stand for?,6,10,what has been the important role of the ?
2,NaN,2.0,"As we navigate the 21st century, AI stands at ...",human cognitive functions,What is the name of the s that can be found in...,What does AI attempt to mimic?,7,10,what technology was approach of the and science ?
3,NaN,3.0,"This includes learning, problem-solving, and d...",problem-solving,What is the name of a school?,"Along with learning, what is one of the proces...",8,10,what are the things that are not self - self - ?
4,NaN,4.0,The evolution of AI has been marked by signifi...,process vast amounts,What is the name of the a term that can be use...,What does machine learning allow machines to do?,4,10,what has been resolved by the studying the the...


In [68]:
df.to_excel('ManualTest_Seq2seq.xlsx', index=False)